In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pandas import set_option
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier

In [2]:
tracks_dataset_grouped_genres = pd.read_csv('dataset/tabular/tracks_dataset_without_outliers.csv')
tracks_dataset_grouped_genres = tracks_dataset_grouped_genres.drop(columns=['Unnamed: 0', 'features_duration_ms'])
# conversion to a list
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: x.replace('[','').replace(']','').replace('\'','').split(','))
# keeping only the first genre
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: x[0])

# dropping useless attributes from the dataset
tracks_dataset_grouped_genres = tracks_dataset_grouped_genres.drop(columns=['id', 'name', 'artists', 'album_name', 'n_bars', 'month', 'day'])

# one-hot encode explicit attribute with 0 and 1 in the same column
tracks_dataset_grouped_genres['explicit'] = tracks_dataset_grouped_genres['explicit'].apply(lambda x: 1 if x == True else 0)

# Convert to lowercase for case-insensitive matching
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].str.lower()

# Grouping similar genres by checking if the keyword is contained in the label
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'rock' if any(kw in x for kw in ['rock', 'punk', 'alternative','garage', 'grunge', 'ska']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'metal' if any(kw in x for kw in ['metal', 'grindcore']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'pop' if any(kw in x for kw in ['pop', 'r&b', 'soul', 'funk', 'reggae', 'r-n-b']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'electronic' if any(kw in x for kw in ['electronic', 'house', 'techno', 'trance','electro','dub', 'dubstep', 'industrial', 'drum-and-bass']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'hip-hop' if any(kw in x for kw in ['hip-hop', 'rap', 'trap', 'grime']) else x)
#tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'jazz' if any(kw in x for kw in ['jazz', 'blues', 'country', 'folk', 'classical']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'national' if any(kw in x for kw in ['british', 'swedish', 'spanish', 'brazil', 'indian', 'iranian', 'german', 'french', 'turkish']) else x)

# Convert back to title case for consistency
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].str.title()

# Print unique genres to verify
print(tracks_dataset_grouped_genres['genre'].unique())

# count records in tracks_dataset_grouped_genres
print(tracks_dataset_grouped_genres['genre'].value_counts())

# remove all records with genre different from electronic, pop, national, rock, metal or jazz
tracks_dataset_grouped_genres = tracks_dataset_grouped_genres[tracks_dataset_grouped_genres['genre'].isin(['Electronic', 'Pop', 'National', 'Rock', 'Metal'])]

print(tracks_dataset_grouped_genres['genre'].value_counts())


['Pop' 'Indie' 'Hip-Hop' 'Songwriter' 'Rock' 'Electronic' 'Country'
 'Dance' 'Metal' 'Latino' 'Folk' 'Latin' 'Piano' 'Emo' 'Chill' 'National'
 'Blues' 'Sad' 'Edm' 'Hardcore' 'Disco' 'Acoustic' 'Dancehall' 'J-Dance'
 'Jazz' 'Ambient' 'Salsa' 'Afrobeat' 'Show-Tunes' 'Groove' 'Anime'
 'Comedy' 'Trip-Hop' 'Children' 'New-Age' 'Disney' 'World-Music' 'Idm'
 'Opera' 'Party' 'Mpb' 'Classical' 'Bluegrass' 'Sleep' 'Happy' 'Samba'
 'Breakbeat' 'Pagode' 'Goth' 'J-Idol' 'Sertanejo' 'Kids' 'Club' 'Malay'
 'Hardstyle' 'Gospel' 'Forro' 'Guitar' 'Honky-Tonk' 'Study' 'Tango'
 'Romance']
genre
Electronic    9811
Pop           9016
National      7493
Rock          7332
Metal         4521
              ... 
Dance          246
Latin          196
Indie          185
Edm            139
Latino         138
Name: count, Length: 62, dtype: int64
genre
Electronic    9811
Pop           9016
National      7493
Rock          7332
Metal         4521
Name: count, dtype: int64


In [3]:

# get the mapping of genres to integers
genres = tracks_dataset_grouped_genres['genre'].unique()
genre_to_int = {genre: i for i, genre in enumerate(genres)}
int_to_genre = {i: genre for i, genre in enumerate(genres)}

# convert genres to integers
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: genre_to_int[x])

# Extract the features and the target where the target is the genre
X = tracks_dataset_grouped_genres.drop(columns=['genre'])
y = tracks_dataset_grouped_genres['genre']

#Extract train and test from the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# normalize the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## XGBoost

In [7]:
params = {
    'objective': ['multi:softmax', 'multi:softprob'],
    'num_class': [5], # number of classes
    'booster': ['gbtree', 'dart', 'gblinear'],
    'learning_rate': [0.01, 0.1],
    'max_depth': [-1, 8, 16 ,32],
    'subsample': [0.5, 0.75, 1.0],
    'sampling_method': ['uniform', 'gradient_based'],
    'lambda': [0.0, 0.5, 1],
    'tree_method': ['auto', 'exact', 'approx', 'hist'],
    'refresh_leaf': [0, 1],
    'process_type': ['default', 'update'],
    'multi_strategy': ['one_output_per_tree', 'multi_output_tree'],
    'random_state': [42]
}
clf = XGBClassifier()

# Grid search
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(clf, param_grid=params, cv=2, n_jobs=-1, verbose=3)
grid_search.fit(X_train, y_train)

print('Best parameters found by grid search are:', grid_search.best_params_)

Fitting 2 folds for each of 27648 candidates, totalling 55296 fits


KeyboardInterrupt: 

In [ ]:

clf = XGBClassifier(objective='binary:logistic',
                    max_depth = 6,
                    learning_rate = 1.0,
                    gamma = 0.0,
                    reg_lambda = 1,
                    tree_method='exact', # 'approx' 
                    random_state=42
                    )
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

# LightGBM

In [ ]:
params = {
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'max_depth': [-1, 8, 16, 32],
    'num_leaves': [31, 62, 128],
    'n_estimators': [100, 200, 300],
    'subsample_for_bin': [200000, 400000, 600000],
    'data_sample_strategy': ['bagging', 'goss'],
    'objective': ['multiclass', 'multiclassova'],
    'num_class': [5],
    'reg_alpha': [0.0, 0.5, 1.0],
    'reg_lambda': [0.0, 0.5, 1.0],
    'learning_rate': [0.01, 0.1],
    'random_state': [42]
}

clf = LGBMClassifier()

# Grid search
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print('Best parameters found by grid search are:', grid_search.best_params_)


In [10]:
clf = LGBMClassifier(boosting_type='gbdt',  #'goss', #'dart'
                     max_depth=-1, # no limit
                     num_leaves=31,
                     n_estimators=100,
                     subsample_for_bin=200000,
                     objective='binary',
                     reg_alpha=0.0, #L1 regularization term on weights
                     reg_lambda=0.0, #L2 regularization term on weights
                     random_state=42
                     )
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.6618205631958087
F1-score [0.63118481 0.59967159 0.77113924 0.76008724 0.54774241]
              precision    recall  f1-score   support

           0       0.61      0.65      0.63      1803
           1       0.58      0.62      0.60      1467
           2       0.77      0.78      0.77      1962
           3       0.75      0.77      0.76       904
           4       0.62      0.49      0.55      1499

    accuracy                           0.66      7635
   macro avg       0.66      0.66      0.66      7635
weighted avg       0.66      0.66      0.66      7635
